<a href="https://colab.research.google.com/github/Akanksha7083/Wrist_Fracture_Detection_Using-MURA-dataset/blob/main/Wrist_Fracture_Detection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install essential libraries
!pip install tensorflow keras numpy pandas matplotlib seaborn scikit-learn streamlit -q
!pip install -q streamlit_ace

# Mount your Google Drive to access uploaded dataset or save models
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 46.2 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/MURA-v1.1/train_labeled_studies.csv', header=None)
valid_df = pd.read_csv('/content/drive/MyDrive/MURA-v1.1/valid_labeled_studies.csv', header=None)

train_df.columns = ['study_path', 'label']
valid_df.columns = ['study_path', 'label']

# Filter for only wrist studies
train_df = train_df[train_df['study_path'].str.contains('wrist', case=False)]
valid_df = valid_df[valid_df['study_path'].str.contains('wrist', case=False)]

print(f"Train wrist studies: {len(train_df)}")
print(f"Valid wrist studies: {len(valid_df)}")


Train wrist studies: 3460
Valid wrist studies: 237


In [ ]:
print(train_df['study_path'].head())  # Check the first few 'study_path' values
print(valid_df['study_path'].head())  # Check the first few 'study_path' values


0    MURA-v1.1/train/XR_WRIST/patient06359/study1_p...
1    MURA-v1.1/train/XR_WRIST/patient06360/study1_p...
2    MURA-v1.1/train/XR_WRIST/patient06361/study1_p...
3    MURA-v1.1/train/XR_WRIST/patient06362/study1_p...
4    MURA-v1.1/train/XR_WRIST/patient06332/study1_p...
Name: study_path, dtype: object
0    MURA-v1.1/valid/XR_WRIST/patient11185/study1_p...
1    MURA-v1.1/valid/XR_WRIST/patient11186/study1_p...
2    MURA-v1.1/valid/XR_WRIST/patient11186/study2_p...
3    MURA-v1.1/valid/XR_WRIST/patient11186/study3_p...
4    MURA-v1.1/valid/XR_WRIST/patient11187/study1_p...
Name: study_path, dtype: object


In [ ]:
import os, glob
import pandas as pd

# ✅ Function to expand study folders into image-level paths
def expand_study_paths(df):
    images, labels, study_paths = [], [], []

    for _, row in df.iterrows():
        image_folder = os.path.join('/content/drive/MyDrive', row['study_path'])
        image_paths = glob.glob(os.path.join(image_folder, '*.png'))

        for path in image_paths:
            images.append(path)
            labels.append(int(row['label']))
            study_paths.append(row['study_path'])

    return pd.DataFrame({
        'image_path': images,
        'study_path': study_paths,
        'label': labels
    })



In [ ]:
# ✅ Expand all study paths into image-level paths
train_expanded = expand_study_paths(train_df)
valid_expanded = expand_study_paths(valid_df)

# 🚀 Use the full dataset (no sampling)
print(train_expanded.head())
print(f"Total train images: {len(train_expanded)}, validation images: {len(valid_expanded)}")


                                          image_path  \
0  /content/drive/MyDrive/MURA-v1.1/train/XR_WRIS...   
1  /content/drive/MyDrive/MURA-v1.1/train/XR_WRIS...   
2  /content/drive/MyDrive/MURA-v1.1/train/XR_WRIS...   
3  /content/drive/MyDrive/MURA-v1.1/train/XR_WRIS...   
4  /content/drive/MyDrive/MURA-v1.1/train/XR_WRIS...   

                                          study_path  label  
0  MURA-v1.1/train/XR_WRIST/patient06359/study1_p...      1  
1  MURA-v1.1/train/XR_WRIST/patient06359/study1_p...      1  
2  MURA-v1.1/train/XR_WRIST/patient06360/study1_p...      1  
3  MURA-v1.1/train/XR_WRIST/patient06360/study1_p...      1  
4  MURA-v1.1/train/XR_WRIST/patient06360/study1_p...      1  
Total train images: 9752, validation images: 659


In [ ]:
from pathlib import Path
import tensorflow as tf

def write_tfrecord(df, output_path):
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)

    with tf.io.TFRecordWriter(output_path) as writer:
        for _, row in df.iterrows():
            img = tf.io.read_file(row['image_path'])

            example = tf.train.Example(features=tf.train.Features(feature={
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img.numpy()])),
                'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[row['label']]))
            }))
            writer.write(example.SerializeToString())

write_tfrecord(train_expanded, '/content/drive/MyDrive/MURA-v1.1/train_wrist.tfrecords')
write_tfrecord(valid_expanded, '/content/drive/MyDrive/MURA-v1.1/valid_wrist.tfrecords')


KeyboardInterrupt: 

In [ ]:
def parse_tfrecord(example):
    desc = {'image': tf.io.FixedLenFeature([], tf.string), 'label': tf.io.FixedLenFeature([], tf.int64)}
    example = tf.io.parse_single_example(example, desc)

    image = tf.image.decode_png(example['image'], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.image.per_image_standardization(image)

    return image, example['label']

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    angle = tf.random.uniform([], -0.2*3.14, 0.2*3.14)
    image = tf.image.rot90(image, tf.cast(angle // (3.14 / 2), tf.int32))
    image = tf.image.resize_with_crop_or_pad(image, 224, 224)
    return image, label

def load_dataset(path, batch_size=32, shuffle=True, augment=False):
    ds = tf.data.TFRecordDataset(path).map(parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle: ds = ds.shuffle(1000)
    if augment: ds = ds.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

train_dataset = load_dataset('/content/drive/MyDrive/MURA-v1.1/train_wrist.tfrecords', augment=True)
valid_dataset = load_dataset('/content/drive/MyDrive/MURA-v1.1/valid_wrist.tfrecords', shuffle=False)


In [ ]:
print(train_df.columns)  # Check column names in train_df
print(valid_df.columns)  # Check column names in valid_df


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Class weight balance
class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_expanded['label'])
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

# Load base model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze layers
for layer in base_model.layers[:150]:
    layer.trainable = False

# Custom classifier head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from datetime import datetime
import os

log_dir = os.path.join("logs", "fit", datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    EarlyStopping(monitor='val_recall', patience=8, restore_best_weights=True, mode='max'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, mode='max'),
    TensorBoard(log_dir=log_dir)
]

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=50,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=2
)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

y_probs = model.predict(valid_dataset).flatten()

y_true = []
for _, labels in valid_dataset:
    y_true.extend(labels.numpy())
y_true = np.array(y_true)

precision, recall, thresholds = precision_recall_curve(y_true, y_probs)

precision_target = 0.8
meeting = np.where(precision[:-1] >= precision_target)[0]
optimal_idx = meeting[0] if len(meeting) > 0 else np.argmax(2 * precision[:-1] * recall[:-1] / (precision[:-1] + recall[:-1] + 1e-9))
optimal_threshold = thresholds[optimal_idx] if len(thresholds) > 0 else 0.5

y_pred = (y_probs > optimal_threshold).astype(int)

print("\nOptimized Classification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['No Fracture', 'Fracture'], yticklabels=['No Fracture', 'Fracture'])
plt.title(f'Confusion Matrix (Threshold: {optimal_threshold:.3f})')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label='PR Curve')
plt.axvline(x=recall[optimal_idx], color='r', linestyle='--', label=f'Threshold = {optimal_threshold:.3f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.grid()
plt.legend()
plt.show()


In [ ]:
print("Class counts:", np.bincount(y_true))

In [ ]:
# Example save path inside your Google Drive
model_save_path = '/content/drive/MyDrive/MURA-v1.1/wrist_fracture_model.h5'


In [ ]:
!pip install streamlit pyngrok tensorflow pillow


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from PIL import Image
import numpy as np

# Configure app
st.set_page_config(page_title="Wrist Fracture Detector", layout="centered")

@st.cache_resource
def load_model():
    return tf.keras.models.load_model('/content/drive/MyDrive/MURA-v1.1/wrist_fracture_model.h5')

def main():
    st.title("🩻 Wrist Fracture Detection")
    uploaded_file = st.file_uploader("Upload wrist X-ray", type=["jpg", "jpeg", "png"])

    if uploaded_file:
        img = Image.open(uploaded_file).convert('RGB')
        st.image(img, caption="Uploaded X-ray", width=300)

        model = load_model()
        img = img.resize((224, 224))
        img_array = np.expand_dims(np.array(img)/255.0, axis=0)

        with st.spinner('Analyzing...'):
            pred = model.predict(img_array)[0][0]

        if pred >  0.35:
            st.error(f" Fracture Detected (Prediction accuracy: {pred:.1%})")
        else:
            st.success(f" No Fracture Detected ((Prediction accuracy: {1-pred:.1%})")

if __name__ == "__main__":
    main()

In [ ]:
!ngrok config add-authtoken YOUR_NGROK_AUTH_TOKEN_HERE
!ngrok config add-authtoken "2vv8F32qzhZn8AHp0NeBULPjQYT_bwPM2WkvMjGJFbdZJfnJ"
!pip install streamlit tensorflow pillow
!npm install -g localtunnel

In [ ]:
!pip install pyngrok streamlit -q

import os
import time
from pyngrok import ngrok
from threading import Thread

# Function to run Streamlit
def run_streamlit():
    os.system("streamlit run app.py --server.port 8501")

# Start Streamlit in background
Thread(target=run_streamlit).start()
time.sleep(5)

# Authenticate ngrok (replace with your own token)
ngrok.set_auth_token("2vv8F32qzhZn8AHp0NeBULPjQYT_bwPM2WkvMjGJFbdZJfnJ")

# Create tunnel
public_url = ngrok.connect(addr="8501", bind_tls=True)
print("Access your app at:", public_url)
